## Original code NON-rotating planet

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

# Add path to the directory containing HERCULES scripts
sys.path.append('/home/kh23622/HERCULES_development/Python_scripts')
from HERCULES_structures import *

def read_hercules_output(file_path):
    """
    Reads the HERCULES binary output file and returns the planet object.
    
    Args:
        file_path (str): Path to the HERCULES output file.
        
    Returns:
        planet (HERCULES_planet): The planet object containing data from the HERCULES output.
    """
    with open(file_path, 'rb') as f:
        parameters = HERCULES_parameters()
        parameters.read_binary(f)
        planet = HERCULES_planet()
        planet.read_binary(f)
    return planet

def shell_volume(r1, r2):
    """
    Calculates the volume of a spherical shell between two radii.
    
    Args:
        r1 (float): Outer radius of the shell.
        r2 (float): Inner radius of the shell.
        
    Returns:
        float: Volume of the spherical shell.
    """
    return (4/3) * np.pi * (r1**3 - r2**3)

def calculate_particles_in_shell(shell_mass, num_particles, total_mass):
    """
    Determines the number of particles to be distributed in a shell based on its mass.
    
    Args:
        shell_mass (float): Mass of the shell.
        num_particles (int): Total number of particles.
        total_mass (float): Total mass of the planet.
        
    Returns:
        int: Number of particles to be placed in the shell.
    """
    num_particles_per_shell = (shell_mass * num_particles) / total_mass
    return int(np.round(num_particles_per_shell))

def generate_particle_coordinates(r1, r2, num_particles):
    """
    Generates random coordinates for particles uniformly distributed in a spherical shell.
    
    Args:
        r1 (float): Outer radius of the shell.
        r2 (float): Inner radius of the shell.
        num_particles (int): Number of particles to generate.
        
    Returns:
        tuple: Arrays of x, y, and z coordinates of the particles.
    """
    # Generate random angles and radii
    phi = np.random.uniform(0, 2 * np.pi, num_particles)
    theta = np.random.uniform(0, np.pi, num_particles)
    u = np.random.uniform(0, 1, num_particles)
    
    # Compute particle radii in the shell
    r = r2 + (r1 - r2) * u**(1/3)

    # Convert spherical coordinates to Cartesian
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    return x, y, z

def equal_area_latitude_stretching(x, y, z, N):
    """
    Stretches particles along latitude to achieve equal area distribution.
    
    Args:
        x (np.array): x-coordinates of particles.
        y (np.array): y-coordinates of particles.
        z (np.array): z-coordinates of particles.
        N (int): Number of particles.
        
    Returns:
        tuple: Stretched x, y, and z coordinates.
    """
    num_collars = int(np.sqrt(N))
    num_regions_per_collar = int(N / num_collars)
    
    a = 0.2
    b = 2
    
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r)
    
    theta_stretched = np.arccos(np.cos(theta) * np.sqrt(1 - a * np.sin(theta) ** 2))
    
    x_stretched = r * np.sin(theta_stretched) * np.cos(np.arctan2(y, x))
    y_stretched = r * np.sin(theta_stretched) * np.sin(np.arctan2(y, x))
    z_stretched = r * np.cos(theta_stretched)
    
    return x_stretched, y_stretched, z_stretched

def rotate_particles(x, y, z):
    """
    Applies a random rotation to particle coordinates.
    
    Args:
        x (np.array): x-coordinates of particles.
        y (np.array): y-coordinates of particles.
        z (np.array): z-coordinates of particles.
        
    Returns:
        tuple: Rotated x, y, and z coordinates.
    """
    theta = np.random.uniform(0, 2 * np.pi)
    phi = np.random.uniform(0, np.pi)
    
    # Rotation matrix for spherical coordinates
    R = np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi)],
        [np.sin(theta), np.cos(theta)*np.cos(phi), -np.cos(theta)*np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    
    rotated_coords = np.dot(R, np.vstack([x, y, z]))
    return rotated_coords[0], rotated_coords[1], rotated_coords[2]

def main():
    start_time = time.time()  # Start timing
    
    # Path to the HERCULES output file
    file_path = "C:\\Users\\Nischayee\\HERCULES\\Tutorial\\Analysis Script\\output\\Non-rotating_L0_N50_Nm200_k12_f021_p10_l0.92_0.1_2_17"
    planet = read_hercules_output(file_path)
    
    total_mass = planet.Mtot
    print(f"Total Mass of the Planet: {total_mass:.5e} kg")
    
    num_particles = 50000  # Total number of particles to distribute
    layers = planet.Nlayer  # Number of layers in the planet
    
    particle_positions = []
    particle_masses = []
    particle_densities = []
    scaling_factors = []  # List to hold scaling factors for each layer

    # Iterate through each layer
    for i in range(layers):
        shell_density = planet.real_rho[i]
        current_radius = planet.layers[i].a
        next_radius = planet.layers[i + 1].a if i < layers - 1 else 0
        
        shell_vol = shell_volume(current_radius, next_radius)
        
        # Initialize scaled shell volume and mass
        scaled_shell_vol = shell_vol
        scaled_shell_mass = shell_density * scaled_shell_vol
        
        hercules_mass = planet.layers[i].M
        hercules_vol = planet.layers[i].Vol
        
        # Compute scaling factor for volume
        if shell_vol > 0:
            volume_scaling_factor = hercules_vol / shell_vol
            scaled_shell_vol = shell_vol * volume_scaling_factor
            scaled_shell_mass = shell_density * scaled_shell_vol
        else:
            volume_scaling_factor = 1
        
        # Store scaling factor
        scaling_factors.append(volume_scaling_factor)
        
        # Compute number of particles for the current shell
        num_shell_particles = calculate_particles_in_shell(scaled_shell_mass, num_particles, total_mass)
        
        if num_shell_particles > 0:
            # Generate and process particle coordinates
            x, y, z = generate_particle_coordinates(current_radius, next_radius, num_shell_particles)
            x, y, z = equal_area_latitude_stretching(x, y, z, num_shell_particles)
            x, y, z = rotate_particles(x, y, z)
            
            # Store positions and densities
            particle_positions.append((x, y, z))
            particle_densities.append(np.full(num_shell_particles, shell_density))
        
    
    # Compute total number of particles
    total_particles = np.sum([
        calculate_particles_in_shell(
            shell_density * shell_volume(
                planet.layers[i].a, 
                planet.layers[i + 1].a if i < layers - 1 else 0
            ) * scaling_factors[i], 
            num_particles, 
            total_mass
        ) 
        for i, shell_density in enumerate([planet.real_rho[i] for i in range(layers)])
    ])

    # Compute the mass per particle
    mass_per_particle = total_mass / total_particles

    # Apply the scaled mass to all particles
    for i in range(len(particle_positions)):
        particle_masses.append(np.full(len(particle_positions[i][0]), mass_per_particle))
    
    # Concatenate all particle properties
    x_all = np.concatenate([p[0] for p in particle_positions])
    y_all = np.concatenate([p[1] for p in particle_positions])
    z_all = np.concatenate([p[2] for p in particle_positions])
    all_masses = np.concatenate(particle_masses)
    all_densities = np.concatenate(particle_densities)

    # Print the sum of all scaled particle masses
    print(f"Sum of all scaled particle masses: {np.sum(all_masses):.5e} kg")
    
    end_time = time.time()  # End timing
    print(f"Time taken: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    main()


Total Mass of the Planet: 5.98796e+24 kg
Sum of all scaled particle masses: 5.98796e+24 kg
Time taken: 0.35 seconds


## ORIGINAL CODE rotating planet

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

# Add path to the directory containing HERCULES scripts
sys.path.append('/home/kh23622/HERCULES_development/Python_scripts')
from HERCULES_structures import *

def read_hercules_output(file_path):
    """
    Reads the HERCULES binary output file and returns the planet object.
    
    Args:
        file_path (str): Path to the HERCULES output file.
        
    Returns:
        HERCULES_planet: The planet object containing data from the HERCULES output.
    """
    with open(file_path, 'rb') as f:
        parameters = HERCULES_parameters()
        parameters.read_binary(f)  # Read HERCULES parameters from the file
        planet = HERCULES_planet()
        planet.read_binary(f)  # Read planet data from the file
    return planet

def shell_volume(r1, r2):
    """
    Calculates the volume of a spherical shell between two radii.
    
    Args:
        r1 (float): Outer radius of the shell.
        r2 (float): Inner radius of the shell.
        
    Returns:
        float: Volume of the spherical shell.
    """
    return (4/3) * np.pi * (r1**3 - r2**3)

def calculate_particles_in_shell(shell_mass, num_particles, total_mass):
    """
    Determines the number of particles to be distributed in a shell based on its mass.
    
    Args:
        shell_mass (float): Mass of the shell.
        num_particles (int): Total number of particles to be distributed.
        total_mass (float): Total mass of the planet.
        
    Returns:
        int: Number of particles to be placed in the shell.
    """
    num_particles_per_shell = (shell_mass * num_particles) / total_mass
    return int(np.round(num_particles_per_shell))

def generate_particle_coordinates(r1, r2, num_particles):
    """
    Generates random coordinates for particles uniformly distributed in a spherical shell.
    
    Args:
        r1 (float): Outer radius of the shell.
        r2 (float): Inner radius of the shell.
        num_particles (int): Number of particles to generate.
        
    Returns:
        tuple: Arrays of x, y, and z coordinates of the particles.
    """
    # Generate random angles and radii for particles
    phi = np.random.uniform(0, 2 * np.pi, num_particles)  # Azimuthal angle
    theta = np.random.uniform(0, np.pi, num_particles)   # Polar angle
    
    # Generate random values to compute particle radii within the shell
    u = np.random.uniform(0, 1, num_particles)
    r = r2 + (r1 - r2) * u**(1/3)  # Radial position in spherical shell

    # Convert spherical coordinates to Cartesian coordinates
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    return x, y, z

def equal_area_latitude_stretching(x, y, z, N):
    """
    Stretches particles along latitude to achieve equal area distribution.
    
    Args:
        x (np.array): x-coordinates of particles.
        y (np.array): y-coordinates of particles.
        z (np.array): z-coordinates of particles.
        N (int): Number of particles.
        
    Returns:
        tuple: Arrays of x, y, and z coordinates after latitude stretching.
    """
    num_collars = int(np.sqrt(N))
    num_regions_per_collar = int(N / num_collars)
    
    # Stretching parameters
    a = 0.2
    b = 2
    
    # Compute spherical coordinates of particles
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r)
    
    # Apply latitude stretching to achieve equal area
    theta_stretched = np.arccos(np.cos(theta) * np.sqrt(1 - a * np.sin(theta) ** 2))
    
    # Convert stretched spherical coordinates back to Cartesian
    x_stretched = r * np.sin(theta_stretched) * np.cos(np.arctan2(y, x))
    y_stretched = r * np.sin(theta_stretched) * np.sin(np.arctan2(y, x))
    z_stretched = r * np.cos(theta_stretched)
    
    return x_stretched, y_stretched, z_stretched

def rotate_particles(x, y, z):
    """
    Applies a random rotation to particle coordinates to simulate random orientation.
    
    Args:
        x (np.array): x-coordinates of particles.
        y (np.array): y-coordinates of particles.
        z (np.array): z-coordinates of particles.
        
    Returns:
        tuple: Arrays of x, y, and z coordinates after rotation.
    """
    # Generate random angles for rotation
    theta = np.random.uniform(0, 2 * np.pi)
    phi = np.random.uniform(0, np.pi)
    
    # Define rotation matrix based on spherical angles
    R = np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi)],
        [np.sin(theta), np.cos(theta)*np.cos(phi), -np.cos(theta)*np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    
    # Rotate particle coordinates
    rotated_coords = np.dot(R, np.vstack([x, y, z]))
    return rotated_coords[0], rotated_coords[1], rotated_coords[2]

def main():
    """
    Main function to process the HERCULES output, generate particles, and handle their properties.
    """
    start_time = time.time()  # Start timing
    
    # Path to the HERCULES output file
    file_path = "C:\\Users\\Nischayee\\HERCULES\\Tutorial\\Analysis Script\\output\\testing_Earth_S5.0c_L1.00857_N50_Nm200_k12_f021_p10_l0.92_0.1_2_17"
    planet = read_hercules_output(file_path)
    
    total_mass = planet.Mtot  # Total mass of the planet
    print(f"Total Mass of the Planet: {total_mass:.5e} kg")
    
    num_particles = 50000  # Total number of particles to distribute
    
    layers = planet.Nlayer  # Number of layers in the planet
    particle_positions = []  # List to store particle positions
    particle_masses = []  # List to store particle masses
    particle_densities = []  # List to store particle densities
    scaling_factors = []  # List to hold scaling factors for each layer

    # Process each layer
    for i in range(layers):
        shell_density = planet.real_rho[i]
        current_radius = planet.layers[i].a
        next_radius = planet.layers[i + 1].a if i < layers - 1 else 0
        
        shell_vol = shell_volume(current_radius, next_radius)
        
        # Initialize scaled shell volume and mass
        scaled_shell_vol = shell_vol
        scaled_shell_mass = shell_density * scaled_shell_vol
        
        hercules_mass = planet.layers[i].M
        hercules_vol = planet.layers[i].Vol
        
        # Compute scaling factor for shell volume
        if shell_vol > 0:
            volume_scaling_factor = hercules_vol / shell_vol
            scaled_shell_vol = shell_vol * volume_scaling_factor
            scaled_shell_mass = shell_density * scaled_shell_vol
        else:
            volume_scaling_factor = 1
        
        # Store scaling factor
        scaling_factors.append(volume_scaling_factor)
        
        # Compute number of particles for the current shell
        num_shell_particles = calculate_particles_in_shell(scaled_shell_mass, num_particles, total_mass)
        
        if num_shell_particles > 0:
            # Generate and process particle coordinates
            x, y, z = generate_particle_coordinates(current_radius, next_radius, num_shell_particles)
            x, y, z = equal_area_latitude_stretching(x, y, z, num_shell_particles)
            x, y, z = rotate_particles(x, y, z)
            
            # Store particle positions and densities
            particle_positions.append((x, y, z))
            particle_densities.append(np.full(num_shell_particles, shell_density))

    # Compute total number of particles
    total_particles = np.sum([
        calculate_particles_in_shell(
            shell_density * shell_volume(
                planet.layers[i].a, 
                planet.layers[i + 1].a if i < layers - 1 else 0
            ) * scaling_factors[i], 
            num_particles, 
            total_mass
        ) 
        for i, shell_density in enumerate([planet.real_rho[i] for i in range(layers)])
    ])

    # Compute the mass per particle
    mass_per_particle = total_mass / total_particles

    # Apply the scaled mass to all particles
    for i in range(len(particle_positions)):
        particle_masses.append(np.full(len(particle_positions[i][0]), mass_per_particle))
    
    # Concatenate all particle properties into single arrays
    x_all = np.concatenate([p[0] for p in particle_positions])
    y_all = np.concatenate([p[1] for p in particle_positions])
    z_all = np.concatenate([p[2] for p in particle_positions])

    all_masses = np.concatenate(particle_masses)
    all_densities = np.concatenate(particle_densities)

    # Print the sum of all scaled particle masses
    print(f"Sum of all scaled particle masses: {np.sum(all_masses):.5e} kg")
    
    end_time = time.time()  # End timing
    print(f"Time taken: {end_time - start_time:.2f} seconds")


if __name__ == "__main__":
    main()


Total Mass of the Planet: 5.98796e+24 kg
Sum of all scaled particle masses: 5.98796e+24 kg
Time taken: 0.44 seconds


## VECTORISED CODE

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

# Add path to the directory containing HERCULES scripts
sys.path.append('/home/kh23622/HERCULES_development/Python_scripts')
from HERCULES_structures import *

def read_hercules_output(file_path):
    """
    Reads the HERCULES binary output file and returns the planet object.
    
    Args:
        file_path (str): Path to the HERCULES output file.
        
    Returns:
        HERCULES_planet: The planet object containing data from the HERCULES output.
    """
    with open(file_path, 'rb') as f:
        parameters = HERCULES_parameters()
        parameters.read_binary(f)  # Read HERCULES parameters from the file
        planet = HERCULES_planet()
        planet.read_binary(f)  # Read planet data from the file
    return planet

def shell_volume(r1, r2):
    """
    Calculates the volume of a spherical shell between two radii.
    
    Args:
        r1 (float): Outer radius of the shell.
        r2 (float): Inner radius of the shell.
        
    Returns:
        float: Volume of the spherical shell.
    """
    return (4/3) * np.pi * (r1**3 - r2**3)

def calculate_particles_in_shell(shell_mass, num_particles, total_mass):
    """
    Determines the number of particles to be distributed in a shell based on its mass.
    
    Args:
        shell_mass (float): Mass of the shell.
        num_particles (int): Total number of particles to be distributed.
        total_mass (float): Total mass of the planet.
        
    Returns:
        int: Number of particles to be placed in the shell.
    """
    num_particles_per_shell = (shell_mass * num_particles) / total_mass
    return int(np.round(num_particles_per_shell))

def generate_particle_coordinates(r1, r2, num_particles):
    """
    Generates random coordinates for particles uniformly distributed in a spherical shell.
    
    Args:
        r1 (float): Outer radius of the shell.
        r2 (float): Inner radius of the shell.
        num_particles (int): Number of particles to generate.
        
    Returns:
        tuple: Arrays of x, y, and z coordinates of the particles.
    """
    # Generate random azimuthal and polar angles
    phi = np.random.uniform(0, 2 * np.pi, num_particles)  # Azimuthal angle
    theta = np.random.uniform(0, np.pi, num_particles)   # Polar angle
    
    # Generate random values to determine particle radii within the shell
    u = np.random.uniform(0, 1, num_particles)
    r = r2 + (r1 - r2) * u**(1/3)  # Radial position in spherical shell

    # Convert spherical coordinates to Cartesian coordinates
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    return x, y, z

def equal_area_latitude_stretching(x, y, z, N):
    """
    Stretches particles along latitude to achieve equal area distribution.
    
    Args:
        x (np.array): x-coordinates of particles.
        y (np.array): y-coordinates of particles.
        z (np.array): z-coordinates of particles.
        N (int): Number of particles.
        
    Returns:
        tuple: Arrays of x, y, and z coordinates after latitude stretching.
    """
    num_collars = int(np.sqrt(N))
    num_regions_per_collar = int(N / num_collars)
    
    # Parameters for latitude stretching
    a = 0.2
    b = 2
    
    # Compute spherical coordinates of particles
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r)
    
    # Apply latitude stretching to achieve equal area
    theta_stretched = np.arccos(np.cos(theta) * np.sqrt(1 - a * np.sin(theta) ** 2))
    
    # Convert stretched spherical coordinates back to Cartesian coordinates
    x_stretched = r * np.sin(theta_stretched) * np.cos(np.arctan2(y, x))
    y_stretched = r * np.sin(theta_stretched) * np.sin(np.arctan2(y, x))
    z_stretched = r * np.cos(theta_stretched)
    
    return x_stretched, y_stretched, z_stretched

def rotate_particles(x, y, z):
    """
    Applies a random rotation to particle coordinates to simulate random orientation.
    
    Args:
        x (np.array): x-coordinates of particles.
        y (np.array): y-coordinates of particles.
        z (np.array): z-coordinates of particles.
        
    Returns:
        tuple: Arrays of x, y, and z coordinates after rotation.
    """
    # Generate random rotation angles
    theta = np.random.uniform(0, 2 * np.pi)
    phi = np.random.uniform(0, np.pi)
    
    # Define rotation matrix based on spherical angles
    R = np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi)],
        [np.sin(theta), np.cos(theta)*np.cos(phi), -np.cos(theta)*np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    
    # Rotate particle coordinates
    rotated_coords = np.dot(R, np.vstack([x, y, z]))
    return rotated_coords[0], rotated_coords[1], rotated_coords[2]

def main():
    """
    Main function to process the HERCULES output, generate particles, and handle their properties.
    """
    start_time = time.time()  # Start timing
    
    # Path to the HERCULES output file
    file_path = "C:\\Users\\Nischayee\\HERCULES\\Tutorial\\Analysis Script\\output\\testing_Earth_S5.0c_L1.00857_N50_Nm200_k12_f021_p10_l0.92_0.1_2_17"
    planet = read_hercules_output(file_path)
    
    total_mass = planet.Mtot  # Total mass of the planet
    
    num_particles = 50000  # Total number of particles to distribute
    
    layers = planet.Nlayer  # Number of layers in the planet
    particle_positions = []  # List to store particle positions
    particle_masses = []  # List to store particle masses
    particle_densities = []  # List to store particle densities
    scaling_factors = []  # List to hold scaling factors for each layer

    # Process each layer
    for i in range(layers):
        shell_density = planet.real_rho[i]
        current_radius = planet.layers[i].a
        next_radius = planet.layers[i + 1].a if i < layers - 1 else 0
        
        shell_vol = shell_volume(current_radius, next_radius)
        
        # Initialize scaled shell volume and mass
        scaled_shell_vol = shell_vol
        scaled_shell_mass = shell_density * scaled_shell_vol
        
        hercules_mass = planet.layers[i].M
        hercules_vol = planet.layers[i].Vol
        
        # Compute scaling factor for shell volume
        if shell_vol > 0:
            volume_scaling_factor = hercules_vol / shell_vol
            scaled_shell_vol = shell_vol * volume_scaling_factor
            scaled_shell_mass = shell_density * scaled_shell_vol
        else:
            volume_scaling_factor = 1
        
        # Store scaling factor
        scaling_factors.append(volume_scaling_factor)
        
        # Compute number of particles for the current shell
        num_shell_particles = calculate_particles_in_shell(scaled_shell_mass, num_particles, total_mass)
        
        if num_shell_particles > 0:
            # Generate and process particle coordinates
            x, y, z = generate_particle_coordinates(current_radius, next_radius, num_shell_particles)
            x, y, z = equal_area_latitude_stretching(x, y, z, num_shell_particles)
            x, y, z = rotate_particles(x, y, z)
            
            # Store particle positions and densities
            particle_positions.append((x, y, z))
            particle_densities.append(np.full(num_shell_particles, shell_density))

    # Compute total number of particles
    total_particles = np.sum([
        calculate_particles_in_shell(
            shell_density * shell_volume(
                planet.layers[i].a, 
                planet.layers[i + 1].a if i < layers - 1 else 0
            ) * scaling_factors[i], 
            num_particles, 
            total_mass
        ) 
        for i, shell_density in enumerate([planet.real_rho[i] for i in range(layers)])
    ])

    # Compute the mass per particle
    mass_per_particle = total_mass / total_particles

    # Apply the scaled mass to all particles
    for i in range(len(particle_positions)):
        particle_masses.append(np.full(len(particle_positions[i][0]), mass_per_particle))
    
    # Concatenate all particle properties into single arrays
    x_all = np.concatenate([p[0] for p in particle_positions])
    y_all = np.concatenate([p[1] for p in particle_positions])
    z_all = np.concatenate([p[2] for p in particle_positions])

    all_masses = np.concatenate(particle_masses)
    all_densities = np.concatenate(particle_densities)

    # Print the sum of all scaled particle masses
    print(f"Sum of all scaled particle masses: {np.sum(all_masses):.5e} kg")
    
    end_time = time.time()  # End timing
    print(f"Time taken: {end_time - start_time:.2f} seconds")


if __name__ == "__main__":
    main()


Sum of all scaled particle masses: 5.98796e+24 kg
Time taken: 0.46 seconds


## NUMBA OPTIMSIED 

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
from numba import njit

sys.path.append('/home/kh23622/HERCULES_development/Python_scripts')
from HERCULES_structures import *

def read_hercules_output(file_path):
    with open(file_path, 'rb') as f:
        parameters = HERCULES_parameters()
        parameters.read_binary(f)
        planet = HERCULES_planet()
        planet.read_binary(f)
    return planet

@njit
def shell_volume(r1, r2):
    return (4/3) * np.pi * (r1**3 - r2**3)

@njit
def calculate_particles_in_shell(shell_mass, num_particles, total_mass):
    num_particles_per_shell = (shell_mass * num_particles) / total_mass
    return int(np.round(num_particles_per_shell))

@njit
def generate_particle_coordinates(r1, r2, num_particles):
    phi = np.random.uniform(0, 2 * np.pi, num_particles)
    theta = np.random.uniform(0, np.pi, num_particles)
    
    u = np.random.uniform(0, 1, num_particles)
    r = r2 + (r1 - r2) * u**(1/3)

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    return x, y, z

@njit
def equal_area_latitude_stretching(x, y, z):
    num_particles = x.size
    a = 0.2
    
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r)
    
    theta_stretched = np.arccos(np.cos(theta) * np.sqrt(1 - a * np.sin(theta) ** 2))
    
    x_stretched = r * np.sin(theta_stretched) * np.cos(np.arctan2(y, x))
    y_stretched = r * np.sin(theta_stretched) * np.sin(np.arctan2(y, x))
    z_stretched = r * np.cos(theta_stretched)
    
    return x_stretched, y_stretched, z_stretched

@njit
def rotate_particles(x, y, z, theta, phi):
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    cos_phi = np.cos(phi)
    sin_phi = np.sin(phi)
    
    R = np.array([
        [cos_theta, -sin_theta*cos_phi, sin_theta*sin_phi],
        [sin_theta, cos_theta*cos_phi, -cos_theta*sin_phi],
        [0, sin_phi, cos_phi]
    ])
    
    x_rot = R[0, 0]*x + R[0, 1]*y + R[0, 2]*z
    y_rot = R[1, 0]*x + R[1, 1]*y + R[1, 2]*z
    z_rot = R[2, 0]*x + R[2, 1]*y + R[2, 2]*z
    
    return x_rot, y_rot, z_rot

def main():
    start_time = time.time()  # Start timing
    file_path = "C:\\Users\\Nischayee\\HERCULES\\Tutorial\\Analysis Script\\output\\testing_Earth_S5.0c_L1.00857_N50_Nm200_k12_f021_p10_l0.92_0.1_2_17"
    planet = read_hercules_output(file_path)
    
    total_mass = planet.Mtot
    print(f"Total Mass of the Planet: {total_mass:.5e} kg")
    
    num_particles = 50000
    
    layers = planet.Nlayer
    particle_positions = []
    particle_masses = []
    particle_densities = []
    scaling_factors = []  # List to hold scaling factors for each layer

    for i in range(layers):
        shell_density = planet.real_rho[i]
        current_radius = planet.layers[i].a
        next_radius = planet.layers[i + 1].a if i < layers - 1 else 0
        
        shell_vol = shell_volume(current_radius, next_radius)
        
        # Initialize scaled_shell_vol and scaled_shell_mass
        scaled_shell_vol = shell_vol
        scaled_shell_mass = shell_density * scaled_shell_vol
        
        hercules_mass = planet.layers[i].M
        hercules_vol = planet.layers[i].Vol
        
        # Compute scaling factor for volume
        if shell_vol > 0:
            volume_scaling_factor = hercules_vol / shell_vol
            scaled_shell_vol = shell_vol * volume_scaling_factor
            scaled_shell_mass = shell_density * scaled_shell_vol
        else:
            volume_scaling_factor = 1
        
        # Store scaling factor
        scaling_factors.append(volume_scaling_factor)
        
        # Compute number of particles
        num_shell_particles = calculate_particles_in_shell(scaled_shell_mass, num_particles, total_mass)
        
        if num_shell_particles > 0:
            x, y, z = generate_particle_coordinates(current_radius, next_radius, num_shell_particles)
            x, y, z = equal_area_latitude_stretching(x, y, z)
            
            # Random rotation angles
            theta = np.random.uniform(0, 2 * np.pi)
            phi = np.random.uniform(0, np.pi)
            
            x, y, z = rotate_particles(x, y, z, theta, phi)
            
            # Store positions
            particle_positions.append((x, y, z))
            
            # Store the density
            particle_densities.append(np.full(num_shell_particles, shell_density))

    
    # Compute total number of particles
    total_particles = np.sum([calculate_particles_in_shell(shell_density * shell_volume(planet.layers[i].a, planet.layers[i + 1].a if i < layers - 1 else 0) * scaling_factors[i], num_particles, total_mass) for i, shell_density in enumerate([planet.real_rho[i] for i in range(layers)])])

    # Scale the mass of each particle
    mass_per_particle = total_mass / total_particles

    # Apply the scaled mass to all particles
    for i in range(len(particle_positions)):
        particle_masses.append(np.full(len(particle_positions[i][0]), mass_per_particle))
    
    # Concatenate all properties
    x_all = np.concatenate([p[0] for p in particle_positions])
    y_all = np.concatenate([p[1] for p in particle_positions])
    z_all = np.concatenate([p[2] for p in particle_positions])

    all_masses = np.concatenate(particle_masses)
    all_densities = np.concatenate(particle_densities)

    # Print the sum of all scaled particle masses
    print(f"Sum of all scaled particle masses: {np.sum(all_masses):.5e} kg")

    end_time = time.time()  # End timing
    print(f"Time taken: {end_time - start_time} seconds")

if __name__ == "__main__":
    main()


Total Mass of the Planet: 5.98796e+24 kg
Sum of all scaled particle masses: 5.98796e+24 kg
Time taken: 4.366816997528076 seconds
